# Exercise - Chapter Llama Index

Follow instructions for compeletion of the exercies

Install requirements

In [14]:
!pip install openai --quiet
!pip install llama_index --quiet
!pip install awscli --quiet
%pip install fs_s3fs --quiet
%pip install s3fs --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Download five documents by clicking the next line of code locally

In [5]:
#! aws s3 cp s3://webage-genaidata/SpaceDocuments/ SpaceDocuments

curl: (1) Protocol "s3" not supported or disabled in libcurl


In [19]:
import pandas as pd

galxy=pd.read_table('s3://btcampdata/SpaceDocuments/Galaxies and Cosmic Structures.txt')
seti=pd.read_table('s3://btcampdata/SpaceDocuments/The Search for Extraterrestrial Life (SETI).txt')
rover=pd.read_table("s3://btcampdata/SpaceDocuments/NASA's Perseverance Rover.txt")
blackholes=pd.read_table('s3://btcampdata/SpaceDocuments/Black Holes.txt')
expanding=pd.read_table('s3://btcampdata/SpaceDocuments/The Expanding Universe.txt')

text_dict={'galxy':galxy,
  'seti':seti,
  'rover':rover,
  'blackholes':blackholes,
  'expanding':expanding}

data_dir = './SpaceDocuments'

## write the data to the disk
for i in text_dict.keys():
    textdoc=text_dict[i].columns[0]
    with open(data_dir+f'/{i}.txt', 'a') as f:
        f.write(textdoc)
    

Here is the question for you:

## TASK 1

You should read manually (by yourself), read those documents/paragraphs that are downloaded from s3 and copy the document text which is the most suitable for the question/prompt below.

NOTE: You can also load files into this notebook and read it from here.

Question:

In [24]:
prompt = "What methods and tools do scientists employ in their pursuit of detecting signals from intelligent life beyond Earth?"

Loading documents in here and printing them:

In [25]:
import os

def read_text_file(file_path, doc_num):
    with open(file_path, 'r') as f:
        print('\n')
        print("DOCUMENT {}".format(doc_num))
        print(f.read())

doc_num = 0

# iterate through all file
for file in os.listdir(data_dir):
    doc_num += 1
    if file.endswith(".txt"):
        file_path = os.path.join(data_dir, file)

        # call read text file function
        read_text_file(file_path, doc_num)




DOCUMENT 4
NASA's Perseverance rover represents a technological marvel and a pinnacle of human achievement in space exploration. Launched as part of the Mars 2020 mission, it successfully touched down on the Martian surface in February 2021. Perseverance's mission is twofold: search for signs of ancient microbial life and collect rock and soil samples for potential future return to Earth. This rover boasts cutting-edge scientific instruments and even a helicopter, named Ingenuity, which conducted historic powered flights on Mars. Perseverance's mission not only contributes to our understanding of Mars' geology and climate but also advances the prospects of human exploration of the Red Planet.


DOCUMENT 5
The notion of the universe's expansion revolutionized our understanding of the cosmos. Rather than remaining static, the universe is in a constant state of growth. Astronomers have discovered compelling evidence supporting this expansion, primarily through the observation of distant

Copy-paste the text which you think can help in answering the question into "DOCUMENT_TEXT" variable below.

In [26]:
DOCUMENT_TEXT = text_dict['seti'].columns[0]
DOCUMENT_TEXT

'The Search for Extraterrestrial Intelligence (SETI) is a captivating scientific endeavor that aims to detect signals or signs of intelligent life beyond our planet. Researchers involved in SETI utilize a diverse array of instruments, including radio telescopes, to scan the cosmos for any anomalous patterns or signals that might indicate the presence of intelligent beings. While humanity has not yet discovered definitive evidence of extraterrestrial life, the quest continues unabated. The tantalizing possibility of encountering life elsewhere in the universe remains one of the most profound questions of our time, motivating scientists to listen attentively to the cosmic silence.'

## TASK 2

Use:
1) OpenAI model to get the answer on our question
2) Question we that we asked in "prompt" variable
3) Text you selected to get the answer from OpenAI model as context for the model from "DOCUMENT_TEXT" variable.

Construct your prompt to OpenAI model in the following way:

In [27]:
final_prompt = prompt +"\n\n Get the answer from the following text: \n\n" + DOCUMENT_TEXT

In [28]:
import json
from openai import OpenAI

In [29]:
api_key =""

## set the key in the environment for llama index
os.environ['OPENAI_API_KEY'] = api_key

client=OpenAI(api_key=api_key)

In [30]:
response = client.completions.create(model='text-davinci-003',
                                    prompt=final_prompt,
                                    temperature = 0.1,
                                    max_tokens = 64)

In [31]:
dict(response.choices[0])['text']

'\n\nMethods and tools employed by scientists in their pursuit of detecting signals from intelligent life beyond Earth include radio telescopes, scanning the cosmos for anomalous patterns or signals, and listening attentively to the cosmic silence.'

## Task 3

Use LlamaIndex and SimpleDirectoryReader to load and vectorize the documents.
Use the same question from the above to get the answer using LlamaIndex this time.

Steps:
1) Load local files and vectorize them using SimpleDirectoryReader
2) Construct Tree Index (you can try other Indexes if they would work in this scenario)
3) Get the answer using query engine from LlamaIndex

Build Tree Index over downloaded documents

In [32]:
from llama_index import TreeIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(data_dir).load_data()
new_index = TreeIndex.from_documents(documents)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Create query engine

In [33]:
query_engine = new_index.as_query_engine()

Query the documents by using the following question (the same as in Task 1) and print the result

In [34]:
prompt = "What methods and tools do scientists employ in their pursuit of detecting signals from intelligent life beyond Earth?"

In [36]:
response = query_engine.query(prompt)
print(response)

Scientists employ a diverse array of instruments, including radio telescopes, in their pursuit of detecting signals from intelligent life beyond Earth. They utilize these tools to scan the cosmos for any anomalous patterns or signals that might indicate the presence of intelligent beings. The quest to detect signals from extraterrestrial life continues unabated, as scientists listen attentively to the cosmic silence in the hopes of encountering life elsewhere in the universe.
